In [8]:
from atgreen import *
from tqdm import tqdm

In [ ]:
PATH=""

In [5]:
os.chdir(PATH)
db_params=pickle.load(open(f"{PATH}/dicts/db_params.pickle", 'rb'))
cities_list=query4table('cities_boundary','public', db_params)['city']

In [31]:
cities_n_rows_dict={}
for city in tqdm(cities_list):
    cities_n_rows_dict[city]=int(query4grid_unmasked(f'{city}', db_params)['y'].max())

for ind, city in enumerate(cities_list):
    print(ind)
    print(city)
    df=query4esa2grid(city, [10,20,30], db_params, 0, 0)
    # Round values
    for VAR in ['green_size', 'size_intersection']:
        df[VAR]=np.round(df[VAR], 2)
    #Rename variables
    df.rename(columns={"green_size":f"{0}_gs", "size_intersection":f"{0}_si"}, inplace=True)
    df['city']=city    
    df['id']=df.apply(lambda x: x['y']+(cities_n_rows_dict[x['city']])*(x['x']-1), axis=1)
    df=df.set_index(['id', 'x','y', 'city'])
    df2psql(df, 'esa2grid', db_params, if_exists='append', index=True, index_label=['id', 'x','y', 'city'] ,schema='esa')